In [1]:
import torch
from torch import arctan2, arcsin, arccos, pi, sin, cos, tensor, sqrt

In [2]:
L1 = 0.067
L2 = 0.213
L3 = 0.210
L4 = 0.094
BASE_LEN = 0.257
BASE_WIDTH = 0.0935

# Последовательность лап: FR, FL, RR, RL

class Legs:
    def __init__(self):
        self._pHip2B = tensor([
            [ L1 + BASE_LEN / 2,  BASE_WIDTH / 2, 0.0],
            [ L1 + BASE_LEN / 2, -BASE_WIDTH / 2, 0.0],
            [-L1 - BASE_LEN / 2,  BASE_WIDTH / 2, 0.0],
            [-L1 - BASE_LEN / 2, -BASE_WIDTH / 2, 0.0]
        ])
        self._sideSign = tensor([1, -1, 1, -1])

    def h2b(self, pos_h):
        return pos_h + self._pHip2B
    
    def b2h(self, pos_b):
        return pos_b - self._pHip2B

    def forward_kinematics(self, q, frame='HIP'):
        if frame not in {'HIP', 'BODY'}:
            raise ValueError("Frame must be HIP or BODY")
                
        l1 = L4 * self._sideSign
        l2 = -L2
        l3 = -L3
        
        s1 = sin(q[:, 0])
        s2 = sin(q[:, 1])
        s3 = sin(q[:, 2])
        c1 = cos(q[:, 0])
        c2 = cos(q[:, 1])
        c3 = cos(q[:, 2])
        
        c23 = c2 * c3 - s2 * s3
        s23 = s2 * c3 + c2 * s3
        
        pEe2H = torch.zeros((4, 3))
        pEe2H[:, 0] = l3 * s23 + l2 * s2
        pEe2H[:, 1] = l3 * s1 * c23 + l1 * c1 + l2 * c2 * s1
        pEe2H[:, 2] = l3 * c1 * c23 - l1 * s1 + l2 * c1 * c2
        
        if frame == 'BODY':
            return self.h2b(pEe2H)
        else:
            return pEe2H
        
    def inverse_kinematics(self, pEe, frame='HIP'):
        if frame not in {'HIP', 'BODY'}:
            raise ValueError("Frame must be HIP or BODY")
        
        if frame == 'BODY':
            pEe2H = pEe - self._pHip2B
        else:
            pEe2H = pEe
            
        px, py, pz = pEe2H[:, 0], pEe2H[:, 1], pEe2H[:, 2]
        
        b2y = L4 * self._sideSign
        b3z = -L2
        b4z = -L3
        
        a = L4
        c = sqrt(px**2 + py**2 + pz**2)
        b = sqrt(c**2 - a**2)
        
        q1 = self.q1_ik(py, pz, b2y)
        q3 = self.q3_ik(b3z, b4z, b)
        q2 = self.q2_ik(q1, q3, px, py, pz, b3z, b4z)
        
        return torch.stack([-q1, q2, q3], dim=1)

    def q1_ik(self, py, pz, l1):
        L = sqrt(py**2 + pz**2 - l1**2)
        return arctan2(pz * l1 + py * L, py * l1 - pz * L)

    def q3_ik(self, b3z, b4z, b):
        temp = (b3z**2 + b4z**2 - b ** 2) / ((2 * abs(b3z * b4z)))
        temp = torch.clamp(temp, -1, 1)
        return -(pi - arccos(temp))

    def q2_ik(self, q1, q3, px, py, pz, b3z, b4z):
        a1 = py * sin(q1) - pz * cos(q1)
        a2 = px
        m1 = b4z * sin(q3)
        m2 = b3z + b4z * cos(q3)
        return arctan2(m1 * a1 + m2 * a2, m1 * a2 - m2 * a1)

    # def get_jacobian(self, q):
    #     l1 = L4 * self._sideSign
    #     l2 = -L2
    #     l3 = -L3
        
    #     s1 = sin(q[:, 0])
    #     s2 = sin(q[:, 1])
    #     s3 = sin(q[:, 2])
    #     c1 = cos(q[:, 0])
    #     c2 = cos(q[:, 1])
    #     c3 = cos(q[:, 2])
        
    #     c23 = c2*c3 - s2*s3
    #     s23 = s2*c3 + c2*s3
        
    #     J = np.zeros((4, 3, 3))
        
    #     # dP/dq1
    #     J[:, 0, 0] = 0
    #     J[:, 1, 0] = -l3 * c1 * c23 - l2 * c1 * c2 - l1 * s1
    #     J[:, 2, 0] = -l3 * s1 * c23 - l2 * c2 * s1 + l1 * c1
        
    #     # dP/dq2
    #     J[:, 0, 1] = l3 * c23 + l2 * c2
    #     J[:, 1, 1] = l3 * s1 * s23 + l2 * s1 * s2
    #     J[:, 2, 1] = -l3 * c1 * s23 - l2 * c1 * s2
        
    #     # dP/dq3
    #     J[:, 0, 2] = l3 * c23
    #     J[:, 1, 2] = l3 * s1 * s23
    #     J[:, 2, 2] = -l3 * c1 * s23
        
    #     return J

    # def calcForceFromTau(self, tau, q):
    #     J = self.get_jacobian(q)
    #     JT = J.transpose(0, 2, 1)
        
    #     try:
    #         # Попытка решения через обратную матрицу
    #         F = np.linalg.inv(JT) @ np.expand_dims(tau, axis=2)
    #     except np.linalg.LinAlgError:
    #         # Если матрица вырождена - используем псевдоинверсию
    #         F = np.linalg.pinv(JT) @ np.expand_dims(tau, axis=2)
        
    #     return np.squeeze(F)

In [3]:
leg = Legs()

In [4]:
# q = tensor([
#     [0.0, 0.8, -1.5],
#     [0.0, 0.8, -1.5],
#     [0.0, 0.8, -1.5],
#     [0.0, 0.8, -1.5]
# ])
q = tensor([
    [-0.0000,  0.7451, -1.5035],
    [-0.0000,  0.7451, -1.5035],
    [-0.0000,  0.7451, -1.5035],
    [-0.0000,  0.7451, -1.5035]
])
pos_h = leg.forward_kinematics(q)
pos_h

tensor([[ 6.1244e-06,  9.4000e-02, -3.0901e-01],
        [ 6.1244e-06, -9.4000e-02, -3.0901e-01],
        [ 6.1244e-06,  9.4000e-02, -3.0901e-01],
        [ 6.1244e-06, -9.4000e-02, -3.0901e-01]])

In [5]:
pos_h = leg.forward_kinematics(q)
pos_h

tensor([[ 6.1244e-06,  9.4000e-02, -3.0901e-01],
        [ 6.1244e-06, -9.4000e-02, -3.0901e-01],
        [ 6.1244e-06,  9.4000e-02, -3.0901e-01],
        [ 6.1244e-06, -9.4000e-02, -3.0901e-01]])

In [6]:
pos_b = leg.forward_kinematics(q, frame='BODY')
pos_b

tensor([[ 0.1955,  0.1407, -0.3090],
        [ 0.1955, -0.1407, -0.3090],
        [-0.1955,  0.1407, -0.3090],
        [-0.1955, -0.1407, -0.3090]])

In [16]:
a = tensor([
    [ 0.1918, -0.1547, -0.3059],
    [ 0.1918,  0.1552, -0.3057],
    [-0.1950, -0.1547, -0.3059],
    [-0.1950,  0.1552, -0.3057]
])
b = tensor([
    [ 0.1955,  0.1407, -0.3090],
    [ 0.1955, -0.1407, -0.3090],
    [-0.1955,  0.1407, -0.3090],
    [-0.1955, -0.1407, -0.3090]
])

a - b

tensor([[-0.0037, -0.2954,  0.0031],
        [-0.0037,  0.2959,  0.0033],
        [ 0.0005, -0.2954,  0.0031],
        [ 0.0005,  0.2959,  0.0033]])

In [9]:
pos_h = torch.tensor([
    [0.0,  0.0940, -0.3090],
    [0.0, -0.0940, -0.3090],
    [0.0,  0.0940, -0.3090],
    [0.0, -0.0940, -0.3090]
])

In [10]:
q = leg.inverse_kinematics(pos_h)
q

tensor([[-0.0000,  0.7451, -1.5035],
        [-0.0000,  0.7451, -1.5035],
        [-0.0000,  0.7451, -1.5035],
        [-0.0000,  0.7451, -1.5035]])

In [100]:
q = leg.inverse_kinematics(pos_b, frame='BODY')
q

tensor([[        nan,         nan, -2.1326e+00],
        [-1.6645e-04,  4.9998e-01, -1.5000e+00],
        [ 1.6180e-04,  8.0008e-01, -1.5002e+00],
        [-1.6180e-04,  8.0008e-01, -1.5002e+00]])

In [101]:
torch.isnan(q)

tensor([[ True,  True, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]])

In [102]:
torch.where(
    torch.isnan(q),
    0.0,
    q
)

tensor([[ 0.0000e+00,  0.0000e+00, -2.1326e+00],
        [-1.6645e-04,  4.9998e-01, -1.5000e+00],
        [ 1.6180e-04,  8.0008e-01, -1.5002e+00],
        [-1.6180e-04,  8.0008e-01, -1.5002e+00]])

In [172]:
L1 = 0.067
L2 = 0.213
L3 = 0.210
L4 = 0.094
BASE_LEN = 0.257
BASE_WIDTH = 0.0935

class LegsIsaac:
    def __init__(self):
        self._pHip2B = tensor([
            [ L1 + BASE_LEN / 2,  BASE_WIDTH / 2, 0.0],
            [ L1 + BASE_LEN / 2, -BASE_WIDTH / 2, 0.0],
            [-L1 - BASE_LEN / 2,  BASE_WIDTH / 2, 0.0],
            [-L1 - BASE_LEN / 2, -BASE_WIDTH / 2, 0.0]
        ])
        self._sideSign = tensor([1, -1, 1, -1])

    def h2b(self, pos_h):
        return pos_h + self._pHip2B
    
    def b2h(self, pos_b):
        return pos_b - self._pHip2B

    def forward_kinematics(self, q, frame='HIP'):
        if frame not in {'HIP', 'BODY'}:
            raise ValueError("Frame must be HIP or BODY")
                
        l1 = L4 * self._sideSign
        l2 = -L2
        l3 = -L3
        
        s1 = sin(q[:, :, 0])
        s2 = sin(q[:, :, 1])
        s3 = sin(q[:, :, 2])
        c1 = cos(q[:, :, 0])
        c2 = cos(q[:, :, 1])
        c3 = cos(q[:, :, 2])
        
        c23 = c2 * c3 - s2 * s3
        s23 = s2 * c3 + c2 * s3
        
        pEe2H = torch.zeros_like(q)
        pEe2H[:, :, 0] = l3 * s23 + l2 * s2
        pEe2H[:, :, 1] = l3 * s1 * c23 + l1 * c1 + l2 * c2 * s1
        pEe2H[:, :, 2] = l3 * c1 * c23 - l1 * s1 + l2 * c1 * c2
        
        if frame == 'BODY':
            return self.h2b(pEe2H)
        else:
            return pEe2H
        
    def inverse_kinematics(self, pEe, frame='HIP'):
        if frame not in {'HIP', 'BODY'}:
            raise ValueError("Frame must be HIP or BODY")
        
        if frame == 'BODY':
            pEe2H = pEe - self._pHip2B
        else:
            pEe2H = pEe
            
        px, py, pz = pEe2H[:, :, 0], pEe2H[:, :, 1], pEe2H[:, :, 2]
        
        b2y = L4 * self._sideSign
        b3z = -L2
        b4z = -L3
        
        a = L4
        c = sqrt(px**2 + py**2 + pz**2)
        b = sqrt(c**2 - a**2)
        
        q1 = self.q1_ik(py, pz, b2y)
        q3 = self.q3_ik(b3z, b4z, b)
        q2 = self.q2_ik(q1, q3, px, py, pz, b3z, b4z)
        
        return torch.stack([-q1, q2, q3], dim=-1)

    def q1_ik(self, py, pz, l1):
        L = sqrt(py**2 + pz**2 - l1**2)
        return arctan2(pz * l1 + py * L, py * l1 - pz * L)

    def q3_ik(self, b3z, b4z, b):
        temp = (b3z**2 + b4z**2 - b ** 2) / (2 * abs(b3z * b4z))
        temp = torch.clamp(temp, -1, 1)
        return -(pi - arccos(temp))

    def q2_ik(self, q1, q3, px, py, pz, b3z, b4z):
        a1 = py * sin(q1) - pz * cos(q1)
        a2 = px
        m1 = b4z * sin(q3)
        m2 = b3z + b4z * cos(q3)
        return arctan2(m1 * a1 + m2 * a2, m1 * a2 - m2 * a1)

    # def get_jacobian(self, q):
    #     l1 = L4 * self._sideSign
    #     l2 = -L2
    #     l3 = -L3
        
    #     s1 = sin(q[:, 0])
    #     s2 = sin(q[:, 1])
    #     s3 = sin(q[:, 2])
    #     c1 = cos(q[:, 0])
    #     c2 = cos(q[:, 1])
    #     c3 = cos(q[:, 2])
        
    #     c23 = c2*c3 - s2*s3
    #     s23 = s2*c3 + c2*s3
        
    #     J = np.zeros((4, 3, 3))
        
    #     # dP/dq1
    #     J[:, 0, 0] = 0
    #     J[:, 1, 0] = -l3 * c1 * c23 - l2 * c1 * c2 - l1 * s1
    #     J[:, 2, 0] = -l3 * s1 * c23 - l2 * c2 * s1 + l1 * c1
        
    #     # dP/dq2
    #     J[:, 0, 1] = l3 * c23 + l2 * c2
    #     J[:, 1, 1] = l3 * s1 * s23 + l2 * s1 * s2
    #     J[:, 2, 1] = -l3 * c1 * s23 - l2 * c1 * s2
        
    #     # dP/dq3
    #     J[:, 0, 2] = l3 * c23
    #     J[:, 1, 2] = l3 * s1 * s23
    #     J[:, 2, 2] = -l3 * c1 * s23
        
    #     return J

    # def calcForceFromTau(self, tau, q):
    #     J = self.get_jacobian(q)
    #     JT = J.transpose(0, 2, 1)
        
    #     try:
    #         # Попытка решения через обратную матрицу
    #         F = np.linalg.inv(JT) @ np.expand_dims(tau, axis=2)
    #     except np.linalg.LinAlgError:
    #         # Если матрица вырождена - используем псевдоинверсию
    #         F = np.linalg.pinv(JT) @ np.expand_dims(tau, axis=2)
        
    #     return np.squeeze(F)

In [173]:
legs = LegsIsaac()

In [174]:
q = tensor([
    [0.0, 0.5, -1.5],
    [0.0, 0.5, -1.5],
    [0.0, 0.8, -1.5],
    [0.0, 0.8, -1.5]
])

In [175]:
q = q.flatten().repeat(10, 1)
q = q.reshape(q.shape[0], 4, 3)
q.shape

torch.Size([10, 4, 3])

In [176]:
pos_h = legs.forward_kinematics(q)

In [177]:
pos_h

tensor([[[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],
         [-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090]],

        [[ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004],


In [178]:
pos_h_sym = torch.zeros_like(pos_h)
pos_h_sym[:, [0, 1]] = pos_h[:, [2, 3]]
pos_h_sym[:, [2, 3]] = pos_h[:, [0, 1]]
pos_h_sym

tensor([[[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],
         [ 0.0746,  0.0940, -0.3004],
         [ 0.0746, -0.0940, -0.3004]],

        [[-0.0175,  0.0940, -0.3090],
         [-0.0175, -0.0940, -0.3090],


In [179]:
pos_h_sym *= tensor([
    [-1, 1, 1],
    [-1, 1, 1],
    [-1, 1, 1],
    [-1, 1, 1],
])

In [180]:
q = legs.inverse_kinematics(pos_h_sym)

In [1]:
import torch

In [7]:
t = torch.randn(12)
t

tensor([-0.6205, -0.2525, -1.2172, -0.3586,  0.5725,  0.0029, -1.6210, -1.3642,
        -2.0167,  0.8616, -0.8789,  0.7144])

In [8]:
t.reshape(4, 3)

tensor([[-0.6205, -0.2525, -1.2172],
        [-0.3586,  0.5725,  0.0029],
        [-1.6210, -1.3642, -2.0167],
        [ 0.8616, -0.8789,  0.7144]])

In [10]:
t.reshape(4, 3).clamp_(
    torch.zeros(3),
    torch.ones(3)
)

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.5725, 0.0029],
        [0.0000, 0.0000, 0.0000],
        [0.8616, 0.0000, 0.7144]])

In [6]:
t

tensor([0.7651, 0.1611, 0.0000, 0.0000, 0.0000, 0.9278, 1.0000, 0.0000, 0.1160,
        0.6003, 0.2038, 0.0000])